# Run AttnCNP Large


In [1]:
cd ..

/private/home/yannd/projects/Neural-Process-Family


In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [3]:
import neuralproc

In [4]:
import submitit
from train_imgs import main, parse_arguments

log_folder="logs/%j"

In [5]:
def repeat_list(l, n):
    return [i for i in l for _ in range(n)]

class Run:
    def checkpoint(self, args):
        """Resubmits the same callable with the same arguments but makes sure continnue from last chckpnt."""
        args.is_continue_train=True 
        return submitit.utils.DelayedSubmission(self, args)
    
    def __call__(self, args):
        job_env = submitit.utils.JobEnvironment()
        args.starting_run = args.starting_run * job_env.num_tasks + job_env.local_rank
        return main(args)

In [18]:
datasets = ["celeba32", "celeba64", "svhn", "mnist", "zs-multi-mnist"]

args = [parse_arguments("{} {} --starting-run {} --batch-size 8 --patience 5 --is-progressbar".format(m, d, s).split())
        for s in range(6)
        for m in ["SelfAttnCNP"] 
        for d in ["zs-multi-mnist"]]

In [19]:
executor=submitit.SlurmExecutor(folder=log_folder, max_num_timeout=3)
executor.update_parameters(num_gpus=1, 
                           time=6,#0*24*3, 
                           partition="dev", 
                           cpus_per_task=10, 
                           mem='32GB',
                           constraint="volta32gb"
                          )

In [20]:
job = executor.map_array(Run(), args)

In [26]:
j = job[0]
job

[SlurmJob<job_id=17816174_0, task_id=0, state="CANCELLED+">,
 SlurmJob<job_id=17816174_1, task_id=0, state="CANCELLED+">,
 SlurmJob<job_id=17816174_2, task_id=0, state="CANCELLED+">,
 SlurmJob<job_id=17816174_3, task_id=0, state="CANCELLED+">,
 SlurmJob<job_id=17816174_4, task_id=0, state="CANCELLED+">,
 SlurmJob<job_id=17816174_5, task_id=0, state="CANCELLED+">]

In [25]:
for j in job:
    j.cancel()

In [15]:
for j in job:
    print("--------------------------------")
    print(j.stdout())

--------------------------------
submitit INFO (2019-09-18 09:54:29,634) - Starting with JobEnvironment(job_id=17816135_0, hostname=learnfair0339, local_rank=0(1), node=0(1), global_rank=0(1))
submitit INFO (2019-09-18 09:54:29,634) - Loading pickle: /private/home/yannd/projects/Neural-Process-Family/logs/17816135_0/17816135_0_submitted.pkl

--- Training zs-multi-mnist/SelfAttnCNP/run_0 ---

submitit ERROR (2019-09-18 09:54:47,139) - Submitted job triggered an exception

--------------------------------
submitit INFO (2019-09-18 09:54:29,422) - Starting with JobEnvironment(job_id=17816135_1, hostname=learnfair0341, local_rank=0(1), node=0(1), global_rank=0(1))
submitit INFO (2019-09-18 09:54:29,423) - Loading pickle: /private/home/yannd/projects/Neural-Process-Family/logs/17816135_1/17816135_1_submitted.pkl

--- Training zs-multi-mnist/SelfAttnCNP/run_1 ---

submitit ERROR (2019-09-18 09:54:46,258) - Submitted job triggered an exception

--------------------------------
submitit INFO (

In [16]:
for j in job:
    print("--------------------------------")
    print(j.stderr())

--------------------------------
INFO:utils.data.imgs:Resizing ZeroShotMultiMNIST ...
INFO:utils.data.imgs:Resizing ZeroShotMultiMNIST ...
/private/home/yannd/projects/Neural-Process-Family/neuralproc/predefined/mlp.py:67: UserWarning: hidden_size=32 smaller than output=128 and input=128. Setting it to 128.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))
/private/home/yannd/projects/Neural-Process-Family/neuralproc/predefined/mlp.py:67: UserWarning: hidden_size=32 smaller than output=256 and input=128. Setting it to 128.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))

  0%|          | 0/3563 [00:00<?, ?it/s]submitit ERROR (2019-09-18 09:54:47,139) - Submitted job triggered an exception
ERROR:submitit:Submitted job triggered an exception
Traceback (most recent call last):
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/p